In [3]:
import os
import sys
import ollama
import google.generativeai as genai
import anthropic
import ollama
import random
import pandas as pd
from tqdm import tqdm
from google.generativeai.types import RequestOptions
from google.api_core import retry
from typing import List, Tuple
import json
import datetime

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import visualize
import pandas as pd
from utils.utils import add_color_to_tags, extract_parts_0, extract_parts_1
import argparse

In [8]:
def read_jsonl_file(filepath):
    data = [] 
    with open(filepath, 'r') as file:
        for line in file:
            json_obj = json.loads(line)  # Parse the JSON data from the line
            data.append(json_obj)  # Add the parsed JSON object to a list
    return data

def get_prompt(prompt_type: str, few_shot_prompt: str, question: str) -> str:
    prompts = {
        "fs": f"{few_shot_prompt}\n{question}",
        "fs_inst": f"{few_shot_prompt}\n{question}\nI want you to answer this question but your explanation should contain references referring back to the information in the question. To do that, first, re-generate the question with proper tags and then generate your answers. The output format is as follow:\n\
            Reformatted Question: \
                Answer:",
        "zs": f"{question}\nI want you to answer this question but your explanation should contain references referring back to the information in the question. To do that, first, re-generate the question with proper tags (<a>, <b>, <c>, etc) for refered information and then generate your answers that also have the tag (<a>, <b>, <c>, etc) for the grounded information. Give your answer by analyzing step by step, and give only numbers in the final answer. The output format is as follow:\n\
            Reformatted Question: \
                Answer:\
                    Final answer:",
        "fs_xml": f"{few_shot_prompt}\n\nRecreate the following question in the style of the correctly formatted examples shown previously. Make sure that your response has all its information inclosed in the proper <tags>. Begin your response with the <key_facts> section. Make sure that every fact in <key_facts> is very concise and contains a very short reference to the <question>. Do not include a <question> section in your response\n\n<question>\n{question}\n</question>",
        "mermaid_get_graph": f"{few_shot_prompt}\n\n Your job is to extract the key facts from a question relevant to answering the question. The facts should be represented in a hierarchal format through a mermaid diagram. Do not create duplicate facts across multiple branches that represent the same information. Create a mermaid diagram that represents the key facts in the following question: \n\n{question}", 
        "mermaid_get_answer": f"{few_shot_prompt}\n\n Your job is to recreate the following question in the style of the correctly formatted examples shown previously. Make sure that your response has all its information inclosed in the proper <tags>.\n\n{question}", 
    }
    return prompts.get(prompt_type, "")

def query_gemini(prompt: str) -> str:
    genai.configure(api_key=API_KEYS['gemini'])
    model = genai.GenerativeModel('gemini-1.5-pro-latest')
    response = model.generate_content(prompt, request_options=RequestOptions(retry=retry.Retry(initial=10, multiplier=2, maximum=60, timeout=60)))
    return response.text

def query_claude(prompt: str) -> str:
    client = anthropic.Anthropic(api_key=API_KEYS['claude'])
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

def query_llm(llm_model: str, ids: List[str], questions: List[str], few_shot_prompt: str, prompt_type: str) -> Tuple[List[str], List[str], List[str]]:
    answers = []
    ids_can_be_answered = []
    questions_can_be_answered = []
    
    for id, q in tqdm(zip(ids, questions)):
        prompt = get_prompt(prompt_type, few_shot_prompt, q)
        # print(prompt)
        try:
            if llm_model == 'gemini':
                answer = query_gemini(prompt)
            elif llm_model == 'claude':
                answer = query_claude(prompt)
            elif llm_model == 'llama3.1':
                answer = ollama.generate(model='llama3.1', prompt=prompt)['response']
                print(answer)
            else:
                raise ValueError(f"Unsupported LLM model: {llm_model}")
            
            answers.append(answer)
            questions_can_be_answered.append(q)
            ids_can_be_answered.append(id)
        except Exception as e:
            print(f"Error processing question {id}: {str(e)}")
            continue
    
    return ids_can_be_answered, questions_can_be_answered, answers

def load_data(data_path: str, sample_size: int = None) -> Tuple[List[str], List[str]]:
    data = read_jsonl_file(data_path)
    print(data_path)
    print(data)
    if sample_size:
        data = random.sample(data, sample_size)
    questions = [x["question"] for x in data]
    ids = [x["id"] for x in data]
    return ids, questions

def load_data_deterministic(data_path: str, sample_size: int = None) -> Tuple[List[str], List[str]]:
    data = read_jsonl_file(data_path)
    print(data_path)
    print(data)
    if sample_size:
        # Sort the data based on a consistent criterion (e.g., 'id' or 'question')
        sorted_data = sorted(data, key=lambda x: x['id'])
        # Take the first 'sample_size' items
        data = sorted_data[:sample_size]
    questions = [x["question"] for x in data]
    ids = [x["id"] for x in data]
    return ids, questions

def load_few_shot_prompt(prompt_path: str) -> str:
    with open(prompt_path, 'r') as file:
        return file.read()

def save_results(save_path: str, ids: List[str], questions: List[str], answers: List[str]):
    df = pd.DataFrame({'id': ids, 'question': questions, 'answer': answers})
    df.to_csv(save_path, index=False)

In [ ]:
time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
project_root = '/Users/log/Github/textual_grounding/'
llm_model = 'llama3.1'
dataset = 'GSM8K'
prompt_type = 'mermaid_get_graph'
# prompt_type = 'fs_inst'
few_shot_txt = 'fewshot_mermaid_graph.txt'
# few_shot_txt = 'fewshot_grounding_prompt_design_1_tin.txt'


data_path = os.path.join(project_root, 'data', dataset, 'test.jsonl')
fewshot_prompt_path = os.path.join(project_root, "prompt", dataset, few_shot_txt)
save_path = os.path.join(project_root, 'logan/results', dataset, 'llama/mermaid', f'{prompt_type}_{llm_model}_{time}.csv')

# Load data and prompt
ids, questions = load_data_deterministic(data_path, sample_size=2)
few_shot_prompt = load_few_shot_prompt(fewshot_prompt_path)


# Query LLM
ids_answered, questions_answered, answers = query_llm(llm_model, ids, questions, few_shot_prompt, prompt_type)

# Save results
save_results(save_path, ids_answered, questions_answered, answers)

In [ ]:
time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
project_root = '/Users/log/Github/textual_grounding/'
llm_model = 'llama3.1'
dataset = 'GSM8K'
prompt_type = 'mermaid_get_answer'
few_shot_txt = 'fewshot_mermaid_full.txt'


# data_path = os.path.join(project_root, 'data', dataset, 'test.jsonl')
data_path = '/Users/log/Github/textual_grounding/logan/results/GSM8K/llama/mermaid/mermaid_get_graph_llama3.1_20240924_001821.csv'

fewshot_prompt_path = os.path.join(project_root, "prompt", dataset, few_shot_txt)
save_path = os.path.join(project_root, 'logan/results', dataset, 'llama/mermaid', f'{prompt_type}_{llm_model}_{time}.csv')

# Load data and prompt
ids, questions = load_data_deterministic(data_path, sample_size=2)
few_shot_prompt = load_few_shot_prompt(fewshot_prompt_path)


# Query LLM
ids_answered, questions_answered, answers = query_llm(llm_model, ids, questions, few_shot_prompt, prompt_type)

# Save results
save_results(save_path, ids_answered, questions_answered, answers)

# XML - visualize

In [83]:
import csv
import re

def extract_parts_1(answer_text):
    """
    Processes the answer text to extract key facts (with numbers), answer reasoning, and the final answer.

    Args:
        answer_text (str): The full answer text containing <key_facts>, <answer_reasoning>, and <final_answer>.

    Returns:
        tuple: (key_facts_list, answer_reasoning, final_answer)
               where key_facts_list is a list of tuples (fact_number, fact_content)
    """
    # Extract key_facts
    key_facts_match = re.search(r'<key_facts>(.*?)</key_facts>', answer_text, re.DOTALL)
    key_facts_content = key_facts_match.group(1).strip() if key_facts_match else ""

    # Extract individual facts with their numbers
    facts = re.findall(r'<fact_(\d+)>(.*?)</fact_\d+>', key_facts_content, re.DOTALL)
    key_facts_list = [(number.strip(), content.strip()) for number, content in facts]

    # Extract answer_reasoning
    reasoning_match = re.search(r'<answer_reasoning>(.*?)</answer_reasoning>', answer_text, re.DOTALL)
    answer_reasoning = reasoning_match.group(1).strip() if reasoning_match else ""

    # Extract final_answer
    final_match = re.search(r'<final_answer>(.*?)</final_answer>', answer_text, re.DOTALL)
    final_answer = final_match.group(1).strip() if final_match else ""

    return key_facts_list, answer_reasoning, final_answer


def add_color_to_tags(text):
    """
    Adds background color to specific tags within the text based on a predefined color mapping.

    Args:
        text (str): The text containing tags like <fact_1>, <fact_2>, etc.

    Returns:
        str: The text with added inline CSS for background colors.
    """
    tag_color_mapping = {
        'fact_1': 'yellow',  
        'fact_2': 'lightblue',
        'fact_3': 'lightgreen',
        'fact_4': 'lightcoral',
        'fact_5': 'lightcyan', 
        'fact_6': 'orange',
    }
    # Iterate over the tag-color mappings
    for tag, color in tag_color_mapping.items():
        # Regex to find the tag and replace it with the same tag having a style attribute
        text = re.sub(
            f'<{tag}>(.*?)</{tag}>',
            f'<{tag} style="background-color: {color};">\\1</{tag}>',
            text,
            flags=re.DOTALL
        )
    return text


def parse_csv_file(file_path):
    """
    Parses the input CSV file and extracts questions and their corresponding answers.

    Args:
        file_path (str): Path to the input CSV file.

    Returns:
        list of tuples: Each tuple contains (question, answer_text).
    """
    qa_pairs = []
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            question = row.get('question', 'No question found.').strip()
            answer_text = row.get('answer', 'No answer found.').strip()
            qa_pairs.append((question, answer_text))
    return qa_pairs


def create_highlight_html(qa_pairs):
    """
    Creates HTML content with highlighted questions, key facts, answer reasoning, and answers.

    Args:
        qa_pairs (list of tuples): Each tuple contains (question, answer_text).

    Returns:
        str: The complete HTML content as a string.
    """
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Question and Answer Highlights</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f0f0f0;
            }
            .container {
                background-color: #ffffff;
                padding: 20px;
                margin-bottom: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            }
            .question {
                font-size: 1.2em;
                margin-bottom: 10px;
            }
            .key-facts {
                margin-bottom: 10px;
            }
            .key-facts ul {
                list-style-type: number;
                padding-left: 20px;
            }
            .key-facts ul li{
                margin-bottom: 4px;
            }
            .answer-reasoning, .final-answer {
                margin-bottom: 10px;
            }
            .highlight {
                background-color: #FFFF00; /* Yellow background for visibility */
                font-weight: bold; /* Bold text for emphasis */
            }
            /* Styles for specific facts */
            fact_1 {
                background-color: yellow;
                font-weight: bold;
            }
            fact_2 {
                background-color: lightblue;
                font-weight: bold;
            }
            fact_3 {
                background-color: lightgreen;
                font-weight: bold;
            }
            fact_4 {
                background-color: lightcoral;
                font-weight: bold;
            }
            fact_5 {
                background-color: lightcyan;
                font-weight: bold;
            }
            fact_6 {
                background-color: orange;
                font-weight: bold;
            }
        </style>
    </head>
    <body>
    <h1>Question and Answer Highlights</h1>
    """
    for i, (question, answer_text) in enumerate(qa_pairs, 1):
        try:
            key_facts, answer_reasoning, final_answer = extract_parts_1(answer_text)
        except Exception as e:
            print(f"Cannot extract parts for question {i}: {e}")
            continue

        # Convert key_facts list to HTML bullet points with "Key fact X:" prefix
        if key_facts:
            key_facts_html = "<ul>\n"
            for fact_number, fact_content in key_facts:
                # Apply color to tags in fact_content
                highlighted_fact = add_color_to_tags(fact_content)
                # Prepend "Key fact X:"
                key_facts_html += f"    <li><fact_{fact_number}>{highlighted_fact}</fact_{fact_number}></li>\n"
            key_facts_html += "</ul>"
        else:
            key_facts_html = "<p>No key facts available.</p>"

        # Apply color to tags in answer_reasoning and final_answer
        highlighted_reasoning = add_color_to_tags(answer_reasoning)
        highlighted_final_answer = add_color_to_tags(final_answer)

        # Build the HTML structure
        html_content += f"<div class='container'>"
        html_content += f"<div class='question'><strong>Question:</strong> {question}</div>"
        html_content += f"<div class='key-facts'><strong>Key Facts:</strong> {key_facts_html}</div>"
        html_content += f"<div class='answer-reasoning'><strong>Answer Reasoning:</strong> {highlighted_reasoning}</div>"
        html_content += f"<div class='final-answer'><strong>Answer:</strong> {highlighted_final_answer}</div>"
        html_content += "</div>\n"

    # Close the HTML tags
    html_content += """
    </body>
    </html>
    """
    return html_content


def main():
    input_file = '/Users/log/Github/textual_grounding/logan/results/GSM8K/llama/test_grounding_answer_prompt_fs_xml_llama3.1.csv'  # Replace with your input CSV file path
    output_file = 'test_grounding_answer_prompt_fs_xml_llama3.1.html'  # Replace with your desired output HTML file path

    # Parse the input CSV file to extract questions and answers
    qa_pairs = parse_csv_file(input_file)

    # Check if any QA pairs were found
    if not qa_pairs:
        print("No question-answer pairs were found in the input file.")
        return

    # Generate the HTML content
    html_content = create_highlight_html(qa_pairs)

    # Write the HTML content to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(html_content)

    print(f"HTML content has been successfully written to {output_file}")


if __name__ == "__main__":
    main()


HTML content has been successfully written to test_grounding_answer_prompt_fs_xml_llama3.1.html


# Tin - visualize

In [84]:
import pandas as pd
import csv
import re

# Define the prompt_type and llm_model as needed
prompt_type = "fs"  # Example value, set accordingly
llm_model = "llama3.1"  # Example value, set accordingly

save_html_path = f"question_answer_highlights_prompt_{prompt_type}_{llm_model}.html"
# df_path = f'logan/results/{dataset}/llama/test_grounding_answer_prompt_{prompt_type}_{llm_model}.csv'
df_path = '/Users/log/Github/textual_grounding/logan/results/GSM8K/llama/test_grounding_answer_prompt_fs_inst_llama3.1.csv'

if prompt_type in ["fs", "fs_inst"]:
    prefix = 'The answer is'
elif prompt_type == "zs":
    prefix = 'Final answer:'

df = pd.read_csv(df_path)
questions = df['question'].tolist()
answers = df['answer'].tolist()

def add_color_to_tags(text):
    """
    Adds background color to specific tags within the text based on a predefined color mapping.

    Args:
        text (str): The text containing tags like <a>, <b>, etc.

    Returns:
        str: The text with added inline CSS for background colors.
    """
    tag_color_mapping = {
        'a': 'yellow',       # You can customize colors as needed
        'b': 'lightblue',
        'c': 'lightgreen',
        'd': 'lightcoral',
        'e': 'lightcyan',
        'f': 'orange',       # Extend as needed
        # Add more tags if necessary
    }
    # Iterate over the tag-color mappings
    for tag, color in tag_color_mapping.items():
        # Regex to find the tag and replace it with the same tag having a style attribute
        text = re.sub(
            f'<{tag}>(.*?)</{tag}>',
            f'<span style="background-color: {color};">{r"\1"}</span>',
            text,
            flags=re.DOTALL
        )
    return text

def parse_csv_file(file_path):
    """
    Parses the input CSV file and extracts questions and their corresponding answers.

    Args:
        file_path (str): Path to the input CSV file.

    Returns:
        list of tuples: Each tuple contains (question, answer_text).
    """
    qa_pairs = []
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            question = row.get('question', 'No question found.').strip()
            answer_text = row.get('answer', 'No answer found.').strip()
            qa_pairs.append((question, answer_text))
    return qa_pairs

def create_highlight_html(qa_pairs):
    """
    Creates HTML content with highlighted questions and answers based on tags.

    Args:
        qa_pairs (list of tuples): Each tuple contains (question, answer_text).

    Returns:
        str: The complete HTML content as a string.
    """
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Question and Answer Highlights</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f0f0f0;
            }
            .container {
                background-color: #ffffff;
                padding: 20px;
                margin-bottom: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            }
            .question {
                font-size: 1.2em;
                margin-bottom: 10px;
            }
            .answer {
                margin-bottom: 10px;
            }
            .highlight {
                background-color: #FFFF00; /* Default highlight color */
                font-weight: bold; /* Bold text for emphasis */
            }
            /* Additional styles for specific tags can be added here if needed */
        </style>
    </head>
    <body>
    <h1>Question and Answer Highlights</h1>
    """
    for i, (question, answer_text) in enumerate(qa_pairs, 1):
        try:
            # Apply color to tags in question and answer
            highlighted_question = add_color_to_tags(question)
            highlighted_answer = add_color_to_tags(answer_text)
        except Exception as e:
            print(f"Cannot process question-answer pair {i}: {e}")
            continue

        # Build the HTML structure
        html_content += f"<div class='container'>"
        html_content += f"<div class='question'><strong>Question:</strong> {highlighted_question}</div>"
        html_content += f"<div class='answer'><strong>Answer:</strong> {highlighted_answer}</div>"
        html_content += "</div>\n"

    # Close the HTML tags
    html_content += """
    </body>
    </html>
    """
    return html_content

def main():
    input_file = df_path  # Use the defined df_path
    output_file = save_html_path  # Use the defined save_html_path

    # Parse the input CSV file to extract questions and answers
    qa_pairs = parse_csv_file(input_file)

    # Check if any QA pairs were found
    if not qa_pairs:
        print("No question-answer pairs were found in the input file.")
        return

    # Generate the HTML content
    html_content = create_highlight_html(qa_pairs)

    # Write the HTML content to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(html_content)

    print(f"HTML content has been successfully written to {output_file}")

if __name__ == "__main__":
    main()


HTML content has been successfully written to question_answer_highlights_prompt_fs_llama3.1.html
